In [3]:
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
from torch.utils.data import DataLoader, random_split
from torch.utils.data.distributed import DistributedSampler
import numpy as np
import pandas as pd
import torch as torch

In [190]:
x_f = torch.rand(500, 456, 10)

In [186]:
x_f.shape

torch.Size([500, 365, 10])

In [191]:
idx = torch.add(torch.arange(365).unsqueeze(axis = 1), torch.arange(91).unsqueeze(axis = 0))

In [192]:
idx

tensor([[  0,   1,   2,  ...,  88,  89,  90],
        [  1,   2,   3,  ...,  89,  90,  91],
        [  2,   3,   4,  ...,  90,  91,  92],
        ...,
        [362, 363, 364,  ..., 450, 451, 452],
        [363, 364, 365,  ..., 451, 452, 453],
        [364, 365, 366,  ..., 452, 453, 454]])

In [193]:
expanded = x_f[:, idx, :] # expanded 

In [194]:
expanded.shape

torch.Size([500, 365, 91, 10])

In [195]:
expanded.view(500, 365, 910).shape # expand

torch.Size([500, 365, 910])

In [198]:
hf1 = nn.Linear(910, 30)(x_f[:, idx, :].view(-1, 365, 910))

In [199]:
hf1.shape

torch.Size([500, 365, 30])

In [200]:
x_t = torch.rand(500, 365, 30)

In [203]:
x_t = x[:, :, 10:40]
x_f = x[:, :, 40:]

In [204]:
x_s = torch.rand(500, 10)

In [205]:
x_s.shape, x_t.shape, x_f.shape

(torch.Size([500, 10]), torch.Size([500, 365, 30]), torch.Size([500, 365, 10]))

In [206]:
s = nn.Linear(10, 30)(x_s)


In [253]:
c1 = F.pad(x_t.permute(0, 2, 1), (0, 0), "constant", 0)
c1 = nn.Conv1d(30, 30, 2, dilation = 1)(c1)
c2 = F.pad(c1, (2, 0), "constant", 0)
c2 = nn.Conv1d(30, 30, 2, dilation = 2)(c2)
c3 = F.pad(c2, (4, 0), "constant", 0)
c3 = nn.Conv1d(30, 30, 2,  dilation = 4)(c3)
c4 = F.pad(c3, (8, 0), "constant", 0)
c4 = nn.Conv1d(30, 30, 2, dilation = 8)(c4)
c5 = F.pad(c4, (16, 0), "constant", 0)
c5 = nn.Conv1d(30, 30, 2, dilation = 16)(c5)
c6 = F.pad(c5, (32, 0), "constant", 0)
c6 = nn.Conv1d(30, 30, 2, dilation = 32)(c6)


In [254]:
t = c6.permute(0, 2, 1)

In [229]:
s.unsqueeze(1).shape

torch.Size([500, 1, 30])

In [255]:
t.shape

torch.Size([500, 364, 30])

In [256]:
encoding = torch.cat((s.unsqueeze(1), t), axis = 1)

In [257]:
encoding.shape

torch.Size([500, 365, 30])

In [258]:
ht = torch.cat((hf1, encoding), axis = 2)

In [259]:
ht.shape

torch.Size([500, 365, 60])

In [260]:
nn.Linear(60, 1820)(ht).shape

torch.Size([500, 365, 1820])

In [246]:
nn.Linear(60, 1820)(ht).view(-1, 365, 91, 20).shape

torch.Size([500, 365, 91, 20])

In [102]:
500*400*50*20

200000000

In [138]:
x_s.unsqueeze(1).repeat(1, 365, 1).shape

torch.Size([500, 365, 10])

In [277]:
nn.Linear(60, 100)(ht).unsqueeze(axis = 2).repeat(1, 1, 91, 1).shape

torch.Size([500, 365, 91, 100])

In [265]:
torch.repeat?

Object `torch.repeat` not found.
